# [期末報告:預測外資持股比例]

## 組員資料:
統計四：劉明翰 105205018
模型搭建、資料爬取

應數碩二：林韋霖 107751006
資料整理、資料爬取

應數四：林銘凱 105701038 模型搭建、整合

財政三：郭曜瑋 106205028
 資料整理、資料爬取

## 資料來源:
https://www.stat.gov.tw/

https://index.ndc.gov.tw/n/zh_tw

https://www.twse.com.tw/zh/

https://www.cnyes.com/

https://www.cbc.gov.tw/tw/mp-1.html

https://finance.yahoo.com/quote/%5EVIX/history/


## 實做流程:
1.抓取資料並依照日期及公司進行整合

2.進行資料處理，建立資料集，每十五天為一單位，預測第16天的持股比例

3.使用tcn模型進行預測

https://medium.com/@cyeninesky3/%E6%99%82%E9%96%93%E5%8D%B7%E7%A9%8D%E7%B6%B2%E7%B5%A1-tcn-%E9%97%9C%E6%96%BC%E5%BE%9E%E9%A2%A8%E6%8E%A7%E9%A0%85%E7%9B%AE%E7%95%B6%E4%B8%AD%E7%9A%84%E5%AD%B8%E7%BF%92-11693d762f5

# 資料前處理

In [ ]:
import pandas as pd
import numpy as np

def preprocess_x(X):
    X.columns = X.loc[0]

    Delete_list_1 = [7, 8, 9, 10, 11, 12, 14, 16, 18, 20, 22, 24, 25, 27, 28, 30, 33, 36, 38, 40, 42]
    Delete_list_2 = list(range(45, 57))
    Delete_list_3 = [60, 61, 64, 65, 66, 68, 69]

    D = Delete_list_1 + Delete_list_2 + Delete_list_3

    ## remove index

    for i in range(len(D)):
        X = X.drop(D[i], axis=0)
    ## remove comma

    for i in range(1,16):
        try:
            X.loc[63][i] = X.loc[63][i].replace(",", "")
        except:
            print(f"comma remove fail in index {i} ")

    ## rescale list
    list_rescale = [1, 2, 3, 4, 5, 6, 32, 57, 58, 63, 67]
    list_n_rescale = [13, 15, 17, 19, 21, 23, 26, 29, 31, 34, 35, 37, 39, 41, 43, 59, 62]
    list_i = [44]

    ## replace to float
    for i in list_rescale + list_n_rescale:
        for j in range(1, 16):
            try:
                X.loc[i][j] = float(X.loc[i][j])
            except:
                print(f"Fail to convert to float{i}")

    ## median normalization(TO DO: exponential transfer?)
    all_issued_median = 458112177.99999994
    MAD = 457654065.8219999
    X.loc[63][1:] = (X.loc[63][1:] - all_issued_median)/(MAD)

    list_rescale.remove(63)
    ## rescale left
    X_re = X.loc[list_rescale]

    ## extract every first nonzero value in each row
    norm = X_re.replace(0, np.nan).bfill(1).iloc[:, 1]

    ## divide
    X_re = X_re.iloc[:, 1:].divide(norm, axis=0)
    X_re = X_re.fillna(0)


    ## for non rescale part
    X_n = X.loc[list_n_rescale]
    X_n = X_n.replace(0, np.nan)
    X_n = X_n.iloc[:, 1:] / 100
    X_n.loc[59] = X_n.loc[59] - 1
    X_n.loc[62] = X_n.loc[62] - 1
    X_n = X_n.fillna(0)

    ## for interest
    X_i = X.loc[[44]].iloc[:, 1:]
    X_i = X_i.fillna(0)

    X = pd.concat([X_re,X_n,X_i],axis = 0)

    return X


if __name__ == '__main__':

    X = pd.read_csv("train1-0.csv")
    Y = pd.read_csv("answer1-0.csv")

    df = preprocess_x(X)


In [ ]:
import os

os.getcwd()

In [ ]:
os.chdir("/Users/math/Desktop/TCN_foreign_invetsment-master/data_before/Xtrain")

In [ ]:
X_L = os.listdir()

In [ ]:
X_ALL = initial
for i in range(1,len(X_L)):
    
    X = pd.read_csv(X_L[i])
    X = preprocess_X.preprocess_x(X)
    X_train = np.transpose(X.values)
    X_train = X_train.reshape(1,15,28)
    X_train = np.asarray(X_train).astype(np.float32)
    
    X_ALL = np.concatenate([X_ALL,X_train])
    
    if i%100==0:
        print(i)

In [ ]:
os.chdir("/Users/math/Desktop/TCN_foreign_invetsment-master")

In [ ]:
import h5py

with h5py.File('X_train.h5', 'w') as hf:
    hf.create_dataset("X_train",  data=X_ALL)

In [ ]:
with h5py.File('X_train.h5', 'r') as hf:
    data = hf['X_train'][:]

data.shape

In [ ]:
def preprocess_y(Y):
    Y_train = np.array([float(Y.loc[66][1])])
    Y_train = Y_train/100
    Y_train = Y_train.reshape(1,1)
    return Y_train

In [ ]:
import os
os.getcwd()

In [ ]:
import os
os.getcwd()

In [ ]:
Y_ALL = initial
for i in range(1,len(Y_L)):
    Y = pd.read_csv(Y_L[i])
    tmp = preprocess_y(Y)
    Y_ALL = np.concatenate([Y_ALL,tmp])
    
    if i%100 == 0:
        print(i)

In [ ]:
import h5py

with h5py.File('Y_train.h5', 'w') as hf:
    hf.create_dataset("Y_train",  data=Y_ALL)

In [ ]:
with h5py.File('Y_train.h5', 'r') as hf:
    data = hf['Y_train'][:]

# TCN模型

This TCN model are based on the repo from https://github.com/philipperemy/keras-tcn

In [4]:
import inspect
from typing import List

from tensorflow.keras import backend as K, Model, Input, optimizers
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation, SpatialDropout1D, Lambda ,Attention
from tensorflow.keras.layers import Layer, Conv1D, Dense, BatchNormalization, LayerNormalization


def is_power_of_two(num): ## 2^
    return num != 0 and ((num & (num - 1)) == 0)


def adjust_dilations(dilations): ## dilation rules
    if all([is_power_of_two(i) for i in dilations]):
        return dilations
    else:
        new_dilations = [2 ** i for i in dilations]
        return new_dilations


class ResidualBlock(Layer):

    def __init__(self,
                 dilation_rate,
                 nb_filters,
                 kernel_size,
                 padding,
                 activation='relu',
                 dropout_rate=0,
                 kernel_initializer='he_normal',
                 use_batch_norm=False,
                 use_layer_norm=False,
                 **kwargs):

        # type: (int, int, int, str, str, float, str, bool, bool, bool, dict) -> None
        """Defines the residual block for the WaveNet TCN

        Args:
            x: The previous layer in the model
            training: boolean indicating whether the layer should behave in training mode or in inference mode
            dilation_rate: The dilation power of 2 we are using for this residual block
            nb_filters: The number of convolutional filters to use in this block
            kernel_size: The size of the convolutional kernel
            padding: The padding used in the convolutional layers, 'same' or 'causal'.
            activation: The final activation used in o = Activation(x + F(x))
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
            kernel_initializer: Initializer for the kernel weights matrix (Conv1D).
            use_batch_norm: Whether to use batch normalization in the residual layers or not.
            use_layer_norm: Whether to use layer normalization in the residual layers or not.
            kwargs: Any initializers for Layer class.
        """

        self.dilation_rate = dilation_rate
        self.nb_filters = nb_filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.use_batch_norm = use_batch_norm
        self.use_layer_norm = use_layer_norm
        self.kernel_initializer = kernel_initializer
        self.layers = []
        self.layers_outputs = []
        self.shape_match_conv = None
        self.res_output_shape = None
        self.final_activation = None

        super(ResidualBlock, self).__init__(**kwargs)

    def _add_and_activate_layer(self, layer):
        """Helper function for building layer

        Args:
            layer: Appends layer to internal layer list and builds it based on the current output
                   shape of ResidualBlocK. Updates current output shape.

        """
        self.layers.append(layer)
        self.layers[-1].build(self.res_output_shape)
        self.res_output_shape = self.layers[-1].compute_output_shape(self.res_output_shape)

    def build(self, input_shape):

        with K.name_scope(self.name):  # name scope used to make sure weights get unique names
            self.layers = []
            self.res_output_shape = input_shape

            for k in range(2):
                name = 'conv1D_{}'.format(k)
                with K.name_scope(name):  # name scope used to make sure weights get unique names
                    self._add_and_activate_layer(Conv1D(filters=self.nb_filters,
                                                        kernel_size=self.kernel_size,
                                                        dilation_rate=self.dilation_rate,
                                                        padding=self.padding,
                                                        name=name,
                                                        kernel_initializer=self.kernel_initializer))

                with K.name_scope('norm_{}'.format(k)):
                    if self.use_batch_norm:
                        self._add_and_activate_layer(BatchNormalization())
                    elif self.use_layer_norm:
                        self._add_and_activate_layer(LayerNormalization())

                self._add_and_activate_layer(Activation('relu'))
                self._add_and_activate_layer(SpatialDropout1D(rate=self.dropout_rate))

            if self.nb_filters != input_shape[-1]:
                # 1x1 conv to match the shapes (channel dimension).
                name = 'matching_conv1D'
                with K.name_scope(name):
                    # make and build this layer separately because it directly uses input_shape
                    self.shape_match_conv = Conv1D(filters=self.nb_filters,
                                                   kernel_size=1,
                                                   padding='same',
                                                   name=name,
                                                   kernel_initializer=self.kernel_initializer)

            else:
                name = 'matching_identity'
                self.shape_match_conv = Lambda(lambda x: x, name=name)

            with K.name_scope(name):
                self.shape_match_conv.build(input_shape)
                self.res_output_shape = self.shape_match_conv.compute_output_shape(input_shape)

            self.final_activation = Activation(self.activation)
            self.final_activation.build(self.res_output_shape)  # probably isn't necessary

            # this is done to force Keras to add the layers in the list to self._layers
            for layer in self.layers:
                self.__setattr__(layer.name, layer)
            self.__setattr__(self.shape_match_conv.name, self.shape_match_conv)
            self.__setattr__(self.final_activation.name, self.final_activation)

            super(ResidualBlock, self).build(input_shape)  # done to make sure self.built is set True

    def call(self, inputs, training=None):
        """
        Returns: A tuple where the first element is the residual model tensor, and the second
                 is the skip connection tensor.
        """
        x = inputs
        self.layers_outputs = [x]
        for layer in self.layers:
            training_flag = 'training' in dict(inspect.signature(layer.call).parameters)
            x = layer(x, training=training) if training_flag else layer(x)
            self.layers_outputs.append(x)
        x2 = self.shape_match_conv(inputs)
        self.layers_outputs.append(x2)
        res_x = layers.add([x2, x])
        self.layers_outputs.append(res_x)

        res_act_x = self.final_activation(res_x)
        self.layers_outputs.append(res_act_x)
        return [res_act_x, x]

    def compute_output_shape(self, input_shape):
        return [self.res_output_shape, self.res_output_shape]


class TCN(Layer):
    """Creates a TCN layer.

        Input shape:
            A tensor of shape (batch_size, timesteps, input_dim).

        Args:
            nb_filters: The number of filters to use in the convolutional layers.
            kernel_size: The size of the kernel to use in each convolutional layer.
            dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
            nb_stacks : The number of stacks of residual blocks to use.
            padding: The padding to use in the convolutional layers, 'causal' or 'same'.
            use_skip_connections: Boolean. If we want to add skip connections from input to each residual blocK.
            return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
            activation: The activation used in the residual blocks o = Activation(x + F(x)).
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
            kernel_initializer: Initializer for the kernel weights matrix (Conv1D).
            use_batch_norm: Whether to use batch normalization in the residual layers or not.
            kwargs: Any other arguments for configuring parent class Layer. For example "name=str", Name of the model.
                    Use unique names when using multiple TCN.

        Returns:
            A TCN layer.
        """

    def __init__(self,
                 nb_filters=64,
                 kernel_size=2,
                 nb_stacks=1,
                 dilations=(1, 2, 4, 8, 16, 32),
                 padding='causal',
                 use_skip_connections=False,
                 dropout_rate=0.0,
                 return_sequences=False,
                 activation='relu',
                 kernel_initializer='he_normal',
                 use_batch_norm=False,
                 use_layer_norm=False,
                 **kwargs):

        self.return_sequences = return_sequences
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        self.activation = activation
        self.padding = padding
        self.kernel_initializer = kernel_initializer
        self.use_batch_norm = use_batch_norm
        self.use_layer_norm = use_layer_norm
        self.skip_connections = []
        self.residual_blocks = []
        self.layers_outputs = []
        self.build_output_shape = None
        self.lambda_layer = None
        self.lambda_ouput_shape = None

        if padding != 'causal' and padding != 'same':
            raise ValueError("Only 'causal' or 'same' padding are compatible for this layer.")

        if not isinstance(nb_filters, int):
            print('An interface change occurred after the version 2.1.2.')
            print('Before: tcn.TCN(x, return_sequences=False, ...)')
            print('Now should be: tcn.TCN(return_sequences=False, ...)(x)')
            print('The alternative is to downgrade to 2.1.2 (pip install keras-tcn==2.1.2).')
            raise Exception()

        # initialize parent class
        super(TCN, self).__init__(**kwargs)

    @property
    def receptive_field(self):
        assert_msg = 'The receptive field formula works only with power of two dilations.'
        assert all([is_power_of_two(i) for i in self.dilations]), assert_msg
        return self.kernel_size * self.nb_stacks * self.dilations[-1]

    def build(self, input_shape):

        # member to hold current output shape of the layer for building purposes
        self.build_output_shape = input_shape

        # list to hold all the member ResidualBlocks
        self.residual_blocks = []
        total_num_blocks = self.nb_stacks * len(self.dilations)
        if not self.use_skip_connections:
            total_num_blocks += 1  # cheap way to do a false case for below

        for s in range(self.nb_stacks):
            for d in self.dilations:
                self.residual_blocks.append(ResidualBlock(dilation_rate=d,
                                                          nb_filters=self.nb_filters,
                                                          kernel_size=self.kernel_size,
                                                          padding=self.padding,
                                                          activation=self.activation,
                                                          dropout_rate=self.dropout_rate,
                                                          use_batch_norm=self.use_batch_norm,
                                                          use_layer_norm=self.use_layer_norm,
                                                          kernel_initializer=self.kernel_initializer,
                                                          name='residual_block_{}'.format(len(self.residual_blocks))))
                # build newest residual block
                self.residual_blocks[-1].build(self.build_output_shape)
                self.build_output_shape = self.residual_blocks[-1].res_output_shape

        # this is done to force keras to add the layers in the list to self._layers
        for layer in self.residual_blocks:
            self.__setattr__(layer.name, layer)

        # Author: @karolbadowski.
        output_slice_index = int(self.build_output_shape.as_list()[1] / 2) if self.padding == 'same' else -1
        self.lambda_layer = Lambda(lambda tt: tt[:, output_slice_index, :])
        self.lambda_ouput_shape = self.lambda_layer.compute_output_shape(self.build_output_shape)

    def compute_output_shape(self, input_shape):
        """
        Overridden in case keras uses it somewhere... no idea. Just trying to avoid future errors.
        """
        if not self.built:
            self.build(input_shape)
        if not self.return_sequences:
            return self.lambda_ouput_shape
        else:
            return self.build_output_shape

    def call(self, inputs, training=None):
        x = inputs
        self.layers_outputs = [x]
        self.skip_connections = []
        for layer in self.residual_blocks:
            x, skip_out = layer(x, training=training)
            self.skip_connections.append(skip_out)
            self.layers_outputs.append(x)

        if self.use_skip_connections:
            x = layers.add(self.skip_connections)
            self.layers_outputs.append(x)

        if not self.return_sequences:
            x = self.lambda_layer(x)
            self.layers_outputs.append(x)
        return x

    def get_config(self):
        """
        Returns the config of a the layer. This is used for saving and loading from a model
        :return: python dictionary with specs to rebuild layer
        """
        config = super(TCN, self).get_config()
        config['nb_filters'] = self.nb_filters
        config['kernel_size'] = self.kernel_size
        config['nb_stacks'] = self.nb_stacks
        config['dilations'] = self.dilations
        config['padding'] = self.padding
        config['use_skip_connections'] = self.use_skip_connections
        config['dropout_rate'] = self.dropout_rate
        config['return_sequences'] = self.return_sequences
        config['activation'] = self.activation
        config['use_batch_norm'] = self.use_batch_norm
        config['use_layer_norm'] = self.use_layer_norm
        config['kernel_initializer'] = self.kernel_initializer
        return config


def compiled_tcn(num_feat,  # type: int
                 num_classes,  # type: int
                 nb_filters,  # type: int
                 kernel_size,  # type: int
                 dilations,  # type: List[int]
                 nb_stacks,  # type: int
                 max_len,  # type: int
                 output_len=1,  # type: int
                 padding='causal',  # type: str
                 use_skip_connections=False,  # type: bool
                 return_sequences=True,
                 regression=False,  # type: bool
                 dropout_rate=0.05,  # type: float
                 name='tcn',  # type: str,
                 kernel_initializer='he_normal',  # type: str,
                 activation='relu',  # type:str,
                 opt='adam',
                 lr=0.002,
                 use_batch_norm=False,
                 use_layer_norm=False):
    # type: (...) -> Model
    """Creates a compiled TCN model for a given task (i.e. regression or classification).
    Classification uses a sparse categorical loss. Please input class ids and not one-hot encodings.

    Args:
        num_feat: The number of features of your input, i.e. the last dimension of: (batch_size, timesteps, input_dim).
        num_classes: The size of the final dense layer, how many classes we are predicting.
        nb_filters: The number of filters to use in the convolutional layers.
        kernel_size: The size of the kernel to use in each convolutional layer.
        dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
        nb_stacks : The number of stacks of residual blocks to use.
        max_len: The maximum sequence length, use None if the sequence length is dynamic.
        padding: The padding to use in the convolutional layers.
        use_skip_connections: Boolean. If we want to add skip connections from input to each residual blocK.
        return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
        regression: Whether the output should be continuous or discrete.
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
        activation: The activation used in the residual blocks o = Activation(x + F(x)).
        name: Name of the model. Useful when having multiple TCN.
        kernel_initializer: Initializer for the kernel weights matrix (Conv1D).
        opt: Optimizer name.
        lr: Learning rate.
        use_batch_norm: Whether to use batch normalization in the residual layers or not.
        use_layer_norm: Whether to use layer normalization in the residual layers or not.
    Returns:
        A compiled keras TCN.
    """

    dilations = adjust_dilations(dilations)

    input_layer = Input(shape=(max_len, num_feat))

    x = TCN(nb_filters, kernel_size, nb_stacks, dilations, padding,
            use_skip_connections, dropout_rate, return_sequences,
            activation, kernel_initializer, use_batch_norm, use_layer_norm,
            name=name)(input_layer)

    print('x.shape=', x.shape)

    def get_opt():
        if opt == 'adam':
            return optimizers.Adam(lr=lr, clipnorm=1.)
        elif opt == 'rmsprop':
            return optimizers.RMSprop(lr=lr, clipnorm=1.)
        else:
            raise Exception('Only Adam and RMSProp are available here')

    if not regression:
        # classification
        x = Dense(num_classes)(x)
        x = Activation('softmax')(x)
        output_layer = x
        model = Model(input_layer, output_layer)

        # https://github.com/keras-team/keras/pull/11373
        # It's now in Keras@master but still not available with pip.
        # TODO remove later.
        def accuracy(y_true, y_pred):
            # reshape in case it's in shape (num_samples, 1) instead of (num_samples,)
            if K.ndim(y_true) == K.ndim(y_pred):
                y_true = K.squeeze(y_true, -1)
            # convert dense predictions to labels
            y_pred_labels = K.argmax(y_pred, axis=-1)
            y_pred_labels = K.cast(y_pred_labels, K.floatx())
            return K.cast(K.equal(y_true, y_pred_labels), K.floatx())

        model.compile(get_opt(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    else:
        # regression
        #x=Attention()([x,x])
        x = Dense(output_len)(x)
        x = Activation('sigmoid')(x) ##change to sigmoid
        output_layer = x
        model = Model(input_layer, output_layer)
        model.compile(get_opt(), loss='mean_squared_error', metrics=['accuracy'])
    print('model.x = {}'.format(input_layer.shape))
    print('model.y = {}'.format(output_layer.shape))
    return model


def tcn_full_summary(model, expand_residual_blocks=True):
    layers = model._layers.copy()  # store existing layers
    model._layers.clear()  # clear layers

    for i in range(len(layers)):
        if isinstance(layers[i], TCN):
            for layer in layers[i]._layers:
                if not isinstance(layer, ResidualBlock):
                    if not hasattr(layer, '__iter__'):
                        model._layers.append(layer)
                else:
                    if expand_residual_blocks:
                        for lyr in layer._layers:
                            if not hasattr(lyr, '__iter__'):
                                model._layers.append(lyr)
                    else:
                        model._layers.append(layer)
        else:
            model._layers.append(layers[i])

    model.summary()  # print summary

    # restore original layers
    model._layers.clear()
    [model._layers.append(lyr) for lyr in layers]


In [ ]:
from tcn import compiled_tcn
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import h5py

In [71]:
with h5py.File('X_train.h5', 'r') as hf:
    X = hf['X_train'][:]

with h5py.File('Y_train.h5', 'r') as hf:
    Y = hf['Y_train'][:]

In [72]:
X.shape

(77526, 15, 28)

In [73]:
X[np.isnan(X)] = 0

In [58]:
Y.shape

(77526, 1)

In [74]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=87)


In [60]:
X_train.shape

(54268, 15, 28)

In [61]:
X_train[np.isnan(X_train)] = 0

In [62]:
Y_train.shape

(54268, 1)

In [5]:
model = compiled_tcn(return_sequences=False,
                             num_feat=28,
                             nb_filters=24,
                             num_classes=0,
                             kernel_size=8,
                             dilations=[2 ** i for i in range(9)],
                             nb_stacks=1,
                             max_len=15,
                             use_skip_connections=True,
                             regression=True,
                             dropout_rate=0,
                             output_len=1) ##opt adam with lr=0.002 default

x.shape= (None, 24)
model.x = (None, 15, 28)
model.y = (None, 1)


In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 15, 28)]          0         
_________________________________________________________________
tcn (TCN)                    (None, 24)                84840     
_________________________________________________________________
dense (Dense)                (None, 1)                 25        
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 84,865
Trainable params: 84,865
Non-trainable params: 0
_________________________________________________________________


In [65]:
model.fit(X_train, Y_train, batch_size=50, epochs=20)

Epoch 1/20
1086/1086 [==============================] - 152s 140ms/step - loss: 0.0277
Epoch 2/20
1086/1086 [==============================] - 158s 146ms/step - loss: 0.0239
Epoch 3/20
1086/1086 [==============================] - 152s 140ms/step - loss: 0.0220
Epoch 4/20
1086/1086 [==============================] - 155s 142ms/step - loss: 0.0208
Epoch 5/20
1086/1086 [==============================] - 152s 140ms/step - loss: 0.0203
Epoch 6/20
1086/1086 [==============================] - 159s 147ms/step - loss: 0.0199
Epoch 7/20
1086/1086 [==============================] - 159s 146ms/step - loss: 0.0198
Epoch 8/20
1086/1086 [==============================] - 157s 145ms/step - loss: 0.0196
Epoch 9/20
1086/1086 [==============================] - 156s 144ms/step - loss: 0.0195
Epoch 10/20
1086/1086 [==============================] - 157s 144ms/step - loss: 0.0193
Epoch 11/20
1086/1086 [==============================] - 155s 142ms/step - loss: 0.0193
Epoch 12/20
1086/1086 [==================

In [76]:
model.fit(X_train, Y_train, batch_size=100, epochs=10)

Epoch 1/10
543/543 [==============================] - 160s 295ms/step - loss: 0.0182
Epoch 2/10
543/543 [==============================] - 161s 297ms/step - loss: 0.0181
Epoch 3/10
543/543 [==============================] - 172s 316ms/step - loss: 0.0181
Epoch 4/10
543/543 [==============================] - 161s 297ms/step - loss: 0.0182
Epoch 5/10
543/543 [==============================] - 176s 324ms/step - loss: 0.0181
Epoch 6/10
543/543 [==============================] - 173s 319ms/step - loss: 0.0181
Epoch 7/10
543/543 [==============================] - 161s 297ms/step - loss: 0.0180
Epoch 8/10
543/543 [==============================] - 167s 307ms/step - loss: 0.0180
Epoch 9/10
543/543 [==============================] - 158s 290ms/step - loss: 0.0180
Epoch 10/10
543/543 [==============================] - 158s 291ms/step - loss: 0.0180


In [77]:
model.fit(X_train, Y_train, batch_size=200, epochs=5)

Epoch 1/5
272/272 [==============================] - 165s 606ms/step - loss: 0.0176
Epoch 2/5
272/272 [==============================] - 155s 570ms/step - loss: 0.0176
Epoch 3/5
272/272 [==============================] - 155s 571ms/step - loss: 0.0176
Epoch 4/5
272/272 [==============================] - 165s 608ms/step - loss: 0.0175
Epoch 5/5
272/272 [==============================] - 165s 607ms/step - loss: 0.0175


In [78]:
model.fit(X_train, Y_train, batch_size=200, epochs=10)

Epoch 1/10
272/272 [==============================] - 163s 599ms/step - loss: 0.0176
Epoch 2/10
272/272 [==============================] - 163s 598ms/step - loss: 0.0175
Epoch 3/10
272/272 [==============================] - 165s 608ms/step - loss: 0.0175
Epoch 4/10
272/272 [==============================] - 164s 602ms/step - loss: 0.0174
Epoch 5/10
272/272 [==============================] - 161s 592ms/step - loss: 0.0175
Epoch 6/10
272/272 [==============================] - 159s 583ms/step - loss: 0.0175
Epoch 7/10
272/272 [==============================] - 159s 584ms/step - loss: 0.0174
Epoch 8/10
272/272 [==============================] - 166s 612ms/step - loss: 0.0174
Epoch 9/10
272/272 [==============================] - 161s 593ms/step - loss: 0.0174
Epoch 10/10
272/272 [==============================] - 170s 624ms/step - loss: 0.0173


In [81]:
model.fit(X_train, Y_train, batch_size=100, epochs=10)

Epoch 1/10
543/543 [==============================] - 164s 302ms/step - loss: 0.0177
Epoch 2/10
543/543 [==============================] - 164s 301ms/step - loss: 0.0177
Epoch 3/10
543/543 [==============================] - 161s 296ms/step - loss: 0.0176
Epoch 4/10
543/543 [==============================] - 169s 311ms/step - loss: 0.0176
Epoch 5/10
543/543 [==============================] - 162s 298ms/step - loss: 0.0176
Epoch 6/10
543/543 [==============================] - 165s 303ms/step - loss: 0.0174
Epoch 7/10
543/543 [==============================] - 163s 301ms/step - loss: 0.0175
Epoch 8/10
543/543 [==============================] - 162s 298ms/step - loss: 0.0173
Epoch 9/10
543/543 [==============================] - 164s 302ms/step - loss: 0.0174
Epoch 10/10
543/543 [==============================] - 166s 306ms/step - loss: 0.0174


In [79]:
model.evaluate(X_test, Y_test)

727/727 [==============================] - 18s 25ms/step - loss: 0.0177


0.017671450972557068

In [82]:
model.save("/Users/math/Desktop/TCN_foreign_invetsment-master")

INFO:tensorflow:Assets written to: /Users/math/Desktop/TCN_foreign_invetsment-master/assets
